In [5]:
tub_names = "/tmp/NavioDonkeyConfig/data/andrew_zen"
new_model_path = "/tmp/NavioDonkeyConfig/models/jup_batchNormalization_128_2_dense"
base_model_path = None
config_path = "/tmp/NavioDonkeyConfig/config.py"
log_path = "/tmp/NavioDonkeyConfig/logs"

In [6]:
import os
import time

from docopt import docopt
import donkeycar as dk

from donkeycar.parts.camera import PiCamera
from donkeycar.parts.actuator import PCA9685_Navio, PWMSteering, PWMThrottle
from donkeycar.parts.datastore import TubGroup, TubWriter
from donkeycar.parts.web_controller import LocalWebController
from donkeycar.parts.clock import Timestamp
from donkeycar.parts.transform import Lambda


In [7]:
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers import Convolution2D
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, BatchNormalization
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard 


class KerasPilot:

    def load(self, model_path):
        self.model = load_model(model_path)

    def shutdown(self):
        pass

    def train(self, train_gen, val_gen,
              saved_model_path, epochs=100, steps=100, train_split=0.8,
              verbose=1, min_delta=.0005, patience=5, use_early_stop=True):
        """
        train_gen: generator that yields an array of images an array of

        """

        # checkpoint to save model after each epoch
        save_best = ModelCheckpoint(saved_model_path,
                                    monitor='val_loss',
                                    verbose=verbose,
                                    save_best_only=True,
                                    mode='min')

        # stop training if the validation error stops improving.
        early_stop = EarlyStopping(monitor='val_loss',
                                   min_delta=min_delta,
                                   patience=patience,
                                   verbose=verbose,
                                   mode='auto')
        
        tb = TensorBoard(log_path)
        callbacks_list = [save_best, tb]

        if use_early_stop:
            callbacks_list.append(early_stop)

        hist = self.model.fit_generator(
            train_gen,
            steps_per_epoch=steps,
            epochs=epochs,
            verbose=1,
            validation_data=val_gen,
            callbacks=callbacks_list,
            validation_steps=int(steps * (1.0 - train_split) / train_split))
        return hist


class KerasLinear(KerasPilot):
    def __init__(self, model=None, num_outputs=None, *args, **kwargs):
        super(KerasLinear, self).__init__(*args, **kwargs)
        if model:
            self.model = model
        elif num_outputs is not None:
            self.model = default_linear()
        else:
            self.model = default_linear()

    def run(self, img_arr):
        img_arr = img_arr.reshape((1,) + img_arr.shape)
        outputs = self.model.predict(img_arr)
        # print(len(outputs), outputs)
        steering = outputs[0]
        throttle = outputs[1]
        return steering[0][0], throttle[0][0]
def default_linear():
    img_in = Input(shape=(120, 160, 3), name='img_in')
    x = img_in

    x = BatchNormalization(axis=1)(x)
    # Convolution2D class name is an alias for Conv2D
    x = Convolution2D(filters=24, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)
    x = BatchNormalization(axis=1)(x)
    x = Convolution2D(filters=32, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)
    x = BatchNormalization(axis=1)(x)
    x = Convolution2D(filters=64, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)
    x = BatchNormalization(axis=1)(x)
    x = Convolution2D(filters=64, kernel_size=(3, 3), strides=(2, 2), activation='relu')(x)
    x = BatchNormalization(axis=1)(x)
    x = Convolution2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu')(x)
    x = BatchNormalization(axis=1)(x)

    x = Flatten(name='flattened')(x)
    x = Dense(units=100, activation='linear')(x)
    x = BatchNormalization(axis=1)(x)
    #x = Dropout(rate=.1)(x)
    x = Dense(units=50, activation='linear')(x)
    x = BatchNormalization(axis=1)(x)
    #x = Dropout(rate=.1)(x)
   
 
    angle_out = Dense(units=1, activation='linear', name='angle_out')(x)

    # continous output of throttle
    throttle_out = Dense(units=1, activation='linear', name='throttle_out')(x)

    model = Model(inputs=[img_in], outputs=[angle_out, throttle_out])

    model.compile(optimizer='adam',
                  loss={'angle_out': 'mean_squared_error',
                        'throttle_out': 'mean_squared_error'},
                  loss_weights={'angle_out': 0.5, 'throttle_out': .5})

    return model


In [8]:
X_keys = ['cam/image_array']
y_keys = ['user/angle', 'user/throttle']

new_model_path = os.path.expanduser(new_model_path)

kl = KerasLinear()
if base_model_path is not None:
    base_model_path = os.path.expanduser(base_model_path)
    kl.load(base_model_path)

print('tub_names', tub_names)
cfg = dk.load_config(config_path)
if not tub_names:
    tub_names = os.path.join(cfg.DATA_PATH, '*')
tubgroup = TubGroup(tub_names)
train_gen, val_gen = tubgroup.get_train_val_gen(X_keys, y_keys,
                                                batch_size=cfg.BATCH_SIZE,
                                                train_frac=cfg.TRAIN_TEST_SPLIT)

total_records = len(tubgroup.df)
total_train = int(total_records * cfg.TRAIN_TEST_SPLIT)
total_val = total_records - total_train
print('train: %d, validation: %d' % (total_train, total_val))
steps_per_epoch = total_train // cfg.BATCH_SIZE
print('steps_per_epoch', steps_per_epoch)
start = time.time()

kl.train(train_gen,
         val_gen,
         saved_model_path=new_model_path,
         steps=steps_per_epoch,
         train_split=cfg.TRAIN_TEST_SPLIT)
end = time.time()
print('Time take', end - start)


tub_names /tmp/NavioDonkeyConfig/data/andrew_zen
loading config file: /tmp/NavioDonkeyConfig/config.py
config loaded
train: 6770, validation: 1693
steps_per_epoch 52
Epoch 1/100
12/12 [==============================] - 6s 509ms/step - loss: 0.2544 - angle_out_loss: 0.3958 - throttle_out_loss: 0.1131

Epoch 00001: val_loss improved from inf to 0.25443, saving model to /tmp/NavioDonkeyConfig/models/jup_batchNormalization_128_2_dense
52/52 [==============================] - 20s 376ms/step - loss: 0.3818 - angle_out_loss: 0.5167 - throttle_out_loss: 0.2469 - val_loss: 0.2544 - val_angle_out_loss: 0.3958 - val_throttle_out_loss: 0.1131
Epoch 2/100
12/12 [==============================] - 6s 496ms/step - loss: 0.1962 - angle_out_loss: 0.3489 - throttle_out_loss: 0.0436

Epoch 00002: val_loss improved from 0.25443 to 0.19622, saving model to /tmp/NavioDonkeyConfig/models/jup_batchNormalization_128_2_dense
52/52 [==============================] - 16s 316ms/step - loss: 0.1273 - angle_out_loss: